In [19]:
import matplotlib.pyplot as plt
from pathlib import Path
import pprint
import numpy as np
import pickle
import os
import sys
sys.path.append( r'C:\Users\Jess\Dropbox\UTokyo\Research\Cellulose\code\AFM analysis\MyImageProcessingLibrary-Ito_develop\MyImageProcessingLibrary-Ito_develop')
sys.path.append(r'C:\Users\Jess\Dropbox\UTokyo\Research\Cellulose\code')
from MyLibrary import imptools
from MyLibrary.BG_Calibrator import BG_Calibrator
from DentAnalyzer import DentAnalyzer
from MyLibrary.ProcessedImageClass import ProcessedImage
from MyLibrary.KinkDetecter import KinkDetecter
from MyLibrary.Segmentater import Segmentater
from MyLibrary.Skeletonizer import Skeletonizer
from MyLibrary.WaveletAnalyzer import WaveletAnalyzer
script_dir = os.path.dirname(os.getcwd())
sys.path.append(script_dir)
from utils.custom_sort import custom_sort

In [20]:
plt.rcParams['font.family'] = 'Segoe UI'
title_font = {'fontname':'Garamond', 'fontsize':14, 'fontweight':'bold'}

In [21]:
# def get_data(file_path): 
#     f = Path(file_path)
#     AFM_data = np.loadtxt(f, skiprows=1)
#     image_size = int(np.sqrt(len(AFM_data)))
#     height_data = AFM_data.reshape((image_size, image_size)) # 2D numpy array
#     # All of our AFM data is square. We did not check whether our modules work for rectanle images.

#     calibrater = BG_Calibrator()
#     segmentater = Segmentater()
#     skeltonizer = Skeletonizer()
#     kinkdetector = KinkDetecter(threshold_angle_from_decomposed_indices=160 * np.pi / 180)
#     dentanalyzer = DentAnalyzer()
#     # The modules should be executed in the following order.
#     image = ProcessedImage(height_data, name=f.stem)
#     calibrater(image)
#     segmentater(image)
#     skeltonizer(image)
#     kinkdetector(image)
#     dentanalyzer(image)
    
#     # Display the processed images.
#     fig, ax = plt.subplots(2, 2, figsize=(10, 10))
#     for a in ax.ravel():
#         a.axis('off')
#     ax[0][0].imshow(image.original_image, cmap='afmhot')
#     ax[0][0].set_title('Original AFM image')
#     ax[0][1].imshow(image.calibrated_image, cmap='afmhot', vmin=-0.5, vmax=4.5)
#     ax[0][1].set_title('Image after background calibration')
#     ax[1][0].imshow(image.binarized_image, cmap='gray')
#     ax[1][0].set_title('Binarized image')
#     ax[1][1].imshow(image.skeleton_image, cmap='gray')
#     ax[1][1].set_title('Skeletonized image')

#     # End points and branch points coordinates can be referred by the following attributes of a ProcessedImage instance.
#     end_points_y, end_points_x = np.where(image.ep)
#     branch_points_y, branch_points_x = np.where(image.bp)
#     ax[1][1].scatter(end_points_x, end_points_y, c='blue', s=10)
#     ax[1][1].scatter(branch_points_x, branch_points_y, c='red', s=10)

#     plt.show()
#     all_kink_coor_x, all_kink_coor_y = image.all_kink_coordinates
#     fig, ax = plt.subplots(1, 1, figsize=(5, 5))
#     ax.axis('off')
#     ax.imshow(image.calibrated_image, cmap='afmhot', vmin=-0.5, vmax=4.5)
#     ax.scatter(all_kink_coor_x, all_kink_coor_y, c='cyan', s=5)
#     ax.set_title('Kink positions')
#     plt.show()
    
#     # Display the decomposed_points used for detection as kink candidates.
#     # The Douglas-Peucker algorithm is used to detect decomposed points.　
#     decomposed_coor_x, decomposed_coor_y = image.decomposed_point_coordinates
#     fig, ax = plt.subplots(1, 1, figsize=(5, 5))
#     ax.axis('off')
#     ax.imshow(image.calibrated_image, cmap='afmhot', vmin=-0.5, vmax=4.5)
#     # 現状では、decomposed_pointsはfiberの端点を含んでいる。使用者は必要に応じて無視してくれ。
#     ax.scatter(decomposed_coor_x, decomposed_coor_y, c='lime', s=5)
#     ax.set_title('Decomposed Points')
#     plt.show()

#     height = list(image.calibrated_image[np.where(image.skeleton_image)])
#     lengths = list()
#     for i in range(1, image.nLabels):
#         y, x, h, w, area = image.data[i]
#         l = imptools.get_length(image.skeleton_image[x: x + w, y: y + h])
#         lengths.append(l)
    
#     return height, lengths, image

In [30]:
def get_data(file_path):
    f = Path(file_path)
    calibrater = BG_Calibrator()
    segmentater = Segmentater()
    skeltonizer = Skeletonizer()
    kinkdetecter = KinkDetecter()  # kink detection method in Fiber class is better
    dentanalyzer = DentAnalyzer()
    processed_images_list = []


    print(f"\nループカウントは{n}です")
    print(f"{f.stem} is processed now...")
    AFM_data = np.loadtxt(f, skiprows=1)
    image_size = int(
        np.sqrt(len(AFM_data))
    )  # the number of pixels that a side of AFM image has
    height_data = AFM_data.reshape((image_size, image_size))

    image = ProcessedImage(height_data, name=f.stem)
    calibrater(image)
    segmentater(image)
    skeltonizer(image)
    kinkdetecter(image)  # kink_detection method in Fiber is better
    dentanalyzer(image)  # 画像処理の結果はimageの属性として保存されているので、適宜参照
    # 必ずこの順番で画像処理しないと動かないので注意。

    processed_images_list.append(image)

    height_all_pixels = imptools.all_pixel_height(processed_images_list)
    length_distribution = imptools.length_distribution(processed_images_list)

    # dentの各カテゴリ,normal, bumpの構成率を格納した辞書
    dent_breakdown_dict = dentanalyzer.calc_breakdown(processed_images_list)
    dent_dl = dentanalyzer.calc_dl(processed_images_list)

    fig, ax = plt.subplots(1, 1)
    for category, (depth, length) in dent_dl.items():
        ax.scatter(length, depth)
    plt.show()

In [22]:
directory = './data/bruker aug 23/'
 
# iterate over files in
# that directory

samples = dict()
for filename in os.listdir(directory):
    if filename.split('.')[-1] == "txt":
        sample_name = filename.split('_')[1].split('.')[0]
        print(sample_name)
        try:
            f = directory + filename
            height, length,  image = get_data(f)
        except Exception as e:
            print(e)
        else:
            sample_data = samples.get(sample_name, None)
            if sample_data:
                new_height = sample_data["height"] + height
                new_length = sample_data["length"] + length
                #new_sample_dents =sample_data["dents"]
                new_image = sample_data["images"]
                new_image.append(image)
              #for dent_type in dents:
                    #new_sample_dents[dent_type] = list(sample_data["dents"][dent_type]) + list(dents[dent_type]) """
                samples[sample_name] = {"height": new_height, "length" :new_length,  "images": new_image}

            else:
                samples[sample_name] = {"height": height,"length": length, "images": [image]}

print(samples)  

p20
'ProcessedImage' object has no attribute 'kink_positions'
p20
'ProcessedImage' object has no attribute 'kink_positions'
p20
'ProcessedImage' object has no attribute 'kink_positions'
p20
'ProcessedImage' object has no attribute 'kink_positions'
p20
'ProcessedImage' object has no attribute 'kink_positions'
p20
'ProcessedImage' object has no attribute 'kink_positions'
p20
'ProcessedImage' object has no attribute 'kink_positions'
p20
'ProcessedImage' object has no attribute 'kink_positions'
p3
'ProcessedImage' object has no attribute 'kink_positions'
p3
'ProcessedImage' object has no attribute 'kink_positions'
p3
'ProcessedImage' object has no attribute 'kink_positions'
p3
'ProcessedImage' object has no attribute 'kink_positions'
p3
'ProcessedImage' object has no attribute 'kink_positions'
p3
'ProcessedImage' object has no attribute 'kink_positions'
p3
'ProcessedImage' object has no attribute 'kink_positions'
p3
'ProcessedImage' object has no attribute 'kink_positions'
p40
'ProcessedIm

In [23]:
      
# for sample_name in samples.keys():

#     lengths = samples.get(sample_name)
#     n = len(lengths)
#     average = np.average(lengths)
#     stddev = np.std(lengths)
#     hist, bins = np.histogram(lengths, bins = 20)

#     filtered_bins = []
#     filtered_hist = []
    
#     for bin_val, count in zip(bins, hist):
#         if count >= 2:
#             filtered_bins.append(bin_val)
#             filtered_hist.append(count)

#     # Calculate the sum of (value * count) for filtered bins
#     sum_value_count = sum(bin_val * count for bin_val, count in zip(filtered_bins, filtered_hist))

#     # Calculate the total count of values in filtered bins
#     total_count = sum(filtered_hist)

#     # Calculate the average
#     average = sum_value_count / total_count
#     n = total_count
#     stddev = np.std(lengths)

#     plt.bar(filtered_bins, filtered_hist, width=np.diff(bins)[0], align='edge')
#     #plt.hist(lengths, bins=20)

#     plt.xlabel('Length [nm]')
    
#     plt.title('Length distribution of ' + sample_name )
#     info = "sample: %s \n n: %d \n average: %.2f \n stddev: %.2f" % (sample_name,  n, average, stddev )
#     textbox_props = dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.5)
#     plt.text(0.5, 0.95, info, transform=plt.gca().transAxes, fontsize=12,
#          verticalalignment='top', horizontalalignment='center', bbox=textbox_props)
#     print(info)

#     plt.savefig("./lengths/" + sample_name + '.png')
#     plt.show()

In [24]:
      
# for sample_name in samples.keys():

#     lengths = samples.get(sample_name)
#     n = len(lengths)
#     average = np.average(lengths)
#     stddev = np.std(lengths)
#     plt.hist(lengths, bins=20)

#     plt.xlabel('Length [nm]')
    
#     plt.title('Length distribution of ' + sample_name )
#     info = "sample: %s \n n: %d \n average: %.2f \n stddev: %.2f" % (sample_name,  n, average, stddev )
#     textbox_props = dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.5)
#     plt.text(0.5, 0.95, info, transform=plt.gca().transAxes, fontsize=12,
#          verticalalignment='top', horizontalalignment='center', bbox=textbox_props)
#     print(info)

#     plt.savefig("./lengths/" + sample_name + '.png')
#     plt.show()

In [25]:
for sample_name in samples.keys():

     data = samples.get(sample_name).get("length")
     # Calculate z-scores for each data point
     z_scores = np.abs((data - np.mean(data)) / np.std(data))
     # Set a z-score threshold for outliers (e.g., z-score > 2)
     z_threshold = 10

     # Filter out outliers
     filtered_data = [value for value, z_score in zip(data, z_scores) if z_score <= z_threshold]

     n_fibrils = len(filtered_data)
     average = np.average(filtered_data)
     stddev = np.std(filtered_data)

     plt.hist(filtered_data, bins=10 ** np.linspace(np.log10(10), np.log10(5000), 50), density = True)
     plt.xscale("log")

     plt.xlabel('Length [nm]')

     plt.title('Length Distribution of ' + sample_name.upper(), **title_font )
     #info = "Sample: %s \n N: %d \n Average: %.2f \n Stddev: %.2f \n z threshold: %.2f"  % (sample_name,  n, average, stddev, z_threshold )
     #info = "Sample: %s \n N: %d \n Average: %.2f \n Stddev: %.2f \n z threshold: %.2f"  % (sample_name,  n_fibrils, average, stddev, z_threshold )
     info = "Sample: %s \n N: %d \n Average: %.2f \n Stddev: %.2f"  % (sample_name.upper(),  n_fibrils, average, stddev )
     textbox_props = dict(boxstyle='square,pad=0.5', facecolor='lightblue', alpha=0.5)
     plt.text(0.8, 0.95, info, transform=plt.gca().transAxes, fontsize=12,
          verticalalignment='top', horizontalalignment='center', bbox=textbox_props)
     print(info)

     plt.savefig("./output/lengths/" + sample_name + '_lengths_nofilter_log.png')
     plt.show()

In [26]:
for sample_name in samples.keys():

     data = samples.get(sample_name).get("height")
     # Calculate z-scores for each data point
     z_scores = np.abs((data - np.mean(data)) / np.std(data))
     # Set a z-score threshold for outliers (e.g., z-score > 2)
     z_threshold = 10

     # Filter out outliers
     filtered_data = [value for value, z_score in zip(data, z_scores) if z_score <= z_threshold]

     n = len(filtered_data)
     average = np.average(filtered_data)
     stddev = np.std(filtered_data)

     plt.hist(filtered_data, bins=20, density = True)
     #plt.xscale("log")

     plt.xlabel('height (nm)')

     plt.title('Height Distribution of ' + sample_name.upper() , **title_font)
     info = "sample: %s \n n: %d \n average: %.2f \n stddev: %.2f \n z threshold: %.2f"  % (sample_name.upper(),  n, average, stddev, z_threshold )
     textbox_props = dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.5)
     plt.text(0.8, 0.95, info, transform=plt.gca().transAxes, fontsize=12,
          verticalalignment='top', horizontalalignment='center', bbox=textbox_props)
     print(info)

     plt.savefig("./outputs/" + sample_name + '_heights.png')
     plt.show()

In [27]:
for sample_name in samples.keys():

     data = samples.get(sample_name).get("dents")

     n_dents = [len(dent)/n_fibrils for dent in data.values()]



     plt.bar(data.keys(), n_dents)

     plt.xlabel('No. of Defects')

     plt.title('Defect Location Distribution of ' + sample_name.upper(), **title_font )
     #info = "Sample: %s \n N: %d \n Average: %.2f \n Stddev: %.2f \n z threshold: %.2f"  % (sample_name,  n, average, stddev, z_threshold )
     #info = "Sample: %s \n N: %d \n Average: %.2f \n Stddev: %.2f \n z threshold: %.2f"  % (sample_name,  n, average, stddev, z_threshold )
     #textbox_props = dict(boxstyle='square,pad=0.5', facecolor='lightblue', alpha=0.5)
     #plt.text(0.8, 0.95, info, transform=plt.gca().transAxes, fontsize=12,
          #verticalalignment='top', horizontalalignment='center', bbox=textbox_props)
     #print(info)

     plt.savefig("./lengths/" + sample_name + '_dents.png')
     plt.show()

In [28]:


for sample_name in samples.keys():

     image_data = samples.get(sample_name).get("images")
     dentanalyzer = DentAnalyzer()
     breakdown_dict = dentanalyzer.calc_breakdown(image_list=image_data)
     dents = breakdown_dict['dent']
     breakdown_dict.pop("dent")
     breakdown_dict.update(dents)
     samples[sample_name]["breakdown"] = breakdown_dict



In [29]:
breakdowns = {}

for sample_name, sample in samples.items():
     breakdowns[sample_name] = list(sample["breakdown"].values())
   
print(breakdowns) 

cats = list(sample["breakdown"].keys())
print(cats)

sample_names = sorted(list(breakdowns.keys()),key=custom_sort)

data_list_rot = [breakdowns[sample] for sample in sample_names]
data_list = list(zip(*data_list_rot[::-1]))
cats_long = {"normal":"Normal", "bump": "Bump", "kink":"Defect at kink", "kinked_end": "Defect at Kinked End", "ep":"Defect at End", "straight": "Defect on Straight Segment"}
print(cats)
print(sample_names)
print(data_list)

fig, ax = plt.subplots()

bottom = np.zeros(len(sample_names))
for i, category in enumerate(cats):

     ax.bar([s.upper() for s in sample_names], data_list[i], label=cats_long[category], bottom=bottom)
     bottom += data_list[i]



ax.set_xlabel('Samples')
ax.set_ylabel('Composition %')

ax.set_title('Defect Distribution', **title_font)
#info = "Sample: %s \n N: %d \n Average: %.2f \n Stddev: %.2f \n z threshold: %.2f"  % (sample_name,  n, average, stddev, z_threshold )
#info = "Sample: %s \n N: %d \n Average: %.2f \n Stddev: %.2f \n z threshold: %.2f"  % (sample_name,  n, average, stddev, z_threshold )
#textbox_props = dict(boxstyle='square,pad=0.5', facecolor='lightblue', alpha=0.5)
#plt.text(0.8, 0.95, info, transform=plt.gca().transAxes, fontsize=12,
     #verticalalignment='top', horizontalalignment='center', bbox=textb
     # 
     # ox_props)
#print(info)
ax.legend()
plt.savefig("./output/defects/" +"defectdist" '.png')
plt.show()

{}


NameError: name 'sample' is not defined